# Nurali Implementation of memory task

Seems to be working somewhat

In [31]:
'''from Training.generator_memory import generator
import numpy as np
from SSM.hippo import hippo_legs, discretize_bilinear
from SSM.helpers import sigmoid, binary_cross_entropy
from SSM.model import SimpleSSM
'''



from generator_memory import generator 
import numpy as np
from SSM.hippo import hippo_legs, discretize_bilinear
from SSM.helpers import sigmoid, binary_cross_entropy
from SSM.mmodel import SimpleSSM

In [83]:
#data parameters


#set data parameters
n_sequence = 1
n_train = 10000
n_test = 1000
time_step = 1
#n_batch

gen = generator(n_sequence, time_step, n_train)

x_train, y_train = gen.seq()

print(x_train[0:10])

gen = generator(n_sequence, time_step, n_test)

x_test, y_test = gen.seq()



[[6. 9. 0.]
 [4. 9. 0.]
 [6. 9. 0.]
 [2. 9. 0.]
 [4. 9. 0.]
 [6. 9. 0.]
 [3. 9. 0.]
 [5. 9. 0.]
 [8. 9. 0.]
 [7. 9. 0.]]


In [15]:
xx = x_train[0].reshape(-1, 1)
xx.shape
yy = y_train[0].reshape(-1,1)
yy.shape

(12, 1)

In [84]:
model = SimpleSSM(input_dim = 1,
        output_dim = 10)
#print(y_train[0])
print(model.A_ct)
print(model.A)
prev = 1000000000
for j in range(100):
    
    epoch_loss = 0.0 
    for i in range(x_train.shape[0]):
        loss, grads = model.loss_and_grads_softmax(x_train[i].reshape(-1, 1), y_train[i].reshape(-1,1))
        model.step(grads, lr = 1e-5)
        epoch_loss += loss
    if j % 10 == 0:
        if epoch_loss/x_train.shape[0] < prev:
            prev = epoch_loss/x_train.shape[0]
        else:
            break
            
    print('average loss= ' + str(epoch_loss/x_train.shape[0]))


print(model.A)



[[ -1.           0.           0.           0.           0.
    0.           0.           0.        ]
 [ -1.73205081  -2.           0.           0.           0.
    0.           0.           0.        ]
 [ -2.23606798  -3.87298335  -3.           0.           0.
    0.           0.           0.        ]
 [ -2.64575131  -4.58257569  -5.91607978  -4.           0.
    0.           0.           0.        ]
 [ -3.          -5.19615242  -6.70820393  -7.93725393  -5.
    0.           0.           0.        ]
 [ -3.31662479  -5.74456265  -7.41619849  -8.77496439  -9.94987437
   -6.           0.           0.        ]
 [ -3.60555128  -6.244998    -8.06225775  -9.53939201 -10.81665383
  -11.95826074  -7.           0.        ]
 [ -3.87298335  -6.70820393  -8.66025404 -10.24695077 -11.61895004
  -12.84523258 -13.96424004  -8.        ]]
[[ 3.33333333e-01 -3.37864825e-16 -2.06854932e-16  1.93156033e-16
   1.87791344e-16 -6.85528257e-17 -3.86372710e-16 -2.22044605e-16]
 [-5.77350269e-01  3.90787006e-16 

In [77]:
x_train[1], y_train[1], model.forward(x_train[1].reshape(-1, 1))

(array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 9., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]),
 (array([[   2.62339966,    1.92393417,    2.04633239,    2.05000411,
             2.1545329 ,    2.1178842 ,    2.10720672,    2.0302966 ,
             1.90721653,    1.78222839],
         [  -2.52792727,   -3.28022248,   -3.11828208,   -3.22469596,
            -3.18081737,   -3.10118025,   -3.13084937,   -3.20244811,
            -3.40191288,   -3.45377811],
         [   6.2051764 ,    5.04252833,    5.03980987,    5.00905464,
             4.97160156,    4.93650795,    5.07991916,    4.98361765,
             4.8395173 ,    4.93391056],
         [  -6.25262686,   -7.34820928,   -7.3258611 ,   -7.47742646,
            -7.55110606,   -7.39409883,   -7.41947572,   -7.57311855,
            -7.51628807,   -7.47265559],
         [  13.05925426,   11.57446173,   11.66604026,   11.5116643 ,
            11.46408569,   11.61800566,   11.53616126,   11.5044134 ,
            11.38792

In [85]:
for i in range(10):
    #print(x_train[i])
    #print(y_train[i])
    logits, _ = model.forward(x_train[i].reshape(-1,1))
    
    pred_digits = np.argmax(logits, axis=1)
    #print(pred_digits)
    if y_train[i].all() == pred_digits.all():
        print(pred_digits, y_train[i])

[0 0 6] [0. 0. 6.]
[0 0 4] [0. 0. 4.]
[0 0 6] [0. 0. 6.]
[0 0 1] [0. 0. 2.]
[0 0 4] [0. 0. 4.]
[0 0 6] [0. 0. 6.]
[0 0 2] [0. 0. 3.]
[0 0 5] [0. 0. 5.]
[0 0 8] [0. 0. 8.]
[0 0 8] [0. 0. 7.]


In [58]:
x_test.shape

(1000, 3)

In [86]:
N = x_test.shape[0]
total_loss = 0.0
total_correct = 0
total_bits = 0

for i in range(N):
    u = x_test[i].reshape(-1, 1)      # (T, 1)
    tgt = y_test[i].reshape(-1,1)    # (T, 1)

    # forward pass only (no grads)
    logits, _ = model.forward(u)

    # loss for this sequence
    pred_digits = np.argmax(logits, axis=1)


    # compute accuracy for this sequence
    #print(tgt.reshape(-1), pred_digits)
    if tgt.reshape(-1)[-1] == pred_digits[-1]:
        total_correct += 1
    
    


accuracy = total_correct / N

print(accuracy)


0.615
